In [1]:
import numpy as np
import sympy as sp

In [17]:
class Node:
    def __init__(self, operation=None, left=None, right=None, value=None):
        self.operation = operation
        self.left = left
        self.right = right
        self.value = value
        
    def evaluate(self):
        if self.operation is None:
            return self.value
        elif self.operation == '+':
            return self.left.evaluate() + self.right.evaluate()
        elif self.operation == '*':
            return self.left.evaluate() * self.right.evaluate()
        
        
    def __repr__(self):
        """노드 정보 보기 쉽게 출력"""
        if self.operation is None:
            return f"({self.value})"
        return f"({self.left} {self.operation} {self.right})"
        

In [21]:
input_nodes = [
    Node(value=3),
    Node(value=2),
    Node(value=1),
    Node(value=7),
    Node(value=5),
    Node(value=4),
]

In [22]:
gate1 = Node(operation='*', left=input_nodes[0], right=input_nodes[1])
gate2 = Node(operation='*', left=gate1, right=Node(operation='+', left=input_nodes[2], right=input_nodes[3]))
gate3 = Node(operation='*', left=Node(operation='+', left=input_nodes[2], right=input_nodes[3]),  right=Node(operation='+', left=input_nodes[4], right=input_nodes[5]))

In [23]:
print("Gate Structure:")
print(f"Gate 1: {gate1}")
print(f"Gate 2: {gate2}")
print(f"Gate 3: {gate3}")

print("\nEvaluations:")
print(f"Gate 1 Result: {gate1.evaluate()}")
print(f"Gate 2 Result: {gate2.evaluate()}")
print(f"Gate 3 Result: {gate3.evaluate()}")

Gate Structure:
Gate 1: ((3) * (2))
Gate 2: (((3) * (2)) * ((1) + (7)))
Gate 3: (((1) + (7)) * ((5) + (4)))

Evaluations:
Gate 1 Result: 6
Gate 2 Result: 48
Gate 3 Result: 72


In [82]:
transcript_table = [3, 2, 1, 7, 5, 4, 6, 48, 72]
gate_list = [gate1, gate2, gate3]


def find_left_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.left.operation == None:
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.left.operation == '+':
        find_left_selector(gate.left.left, gate_idx, c, selector)
        find_left_selector(gate.left.right, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def find_right_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.right.operation == None:
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.right.operation == '+':
        find_right_selector(gate.right.right, gate_idx, c, selector)
        find_right_selector(gate.right.left, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def gate_selector(gate_list, direction, transcript_table):
    polynomial_table = []
    if direction == 'left':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_left_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    elif direction == 'right':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_right_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    return polynomial_table


In [83]:
gate_selector(gate_list, 'left', transcript_table)

gate_idx: 0, c: 3, selector: [1, 0, 0]
gate_idx: 2, c: 1, selector: [0, 0, 1]
gate_idx: 2, c: 7, selector: [0, 0, 1]
gate_idx: 1, c: 6, selector: [0, 1, 0]


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [84]:
gate_selector(gate_list, 'right', transcript_table)

gate_idx: 0, c: 2, selector: [1, 0, 0]
gate_idx: 1, c: 1, selector: [0, 1, 0]
gate_idx: 1, c: 7, selector: [0, 1, 0]
gate_idx: 2, c: 5, selector: [0, 0, 1]
gate_idx: 2, c: 4, selector: [0, 0, 1]


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [115]:
def find_selector(gate, gate_idx, c, selector, direction):
    """Left 또는 Right Selector Polynomial을 찾는 함수"""
    if gate is None:
        return
    
    # 리프 노드라면 값을 확인
    if gate.operation is None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
        return
    
    # 왼쪽 / 오른쪽 / 출력 선택
    if direction == 'left':
        target = gate.left 
    elif direction == 'right':
        target = gate.right
    elif direction == 'output':
        target = gate

    # 리프 값이 맞는 경우
    if target and target.operation is None and target.evaluate() == c:
        selector[gate_idx] = 1
        return

    # 덧셈 노드인 경우 재귀 호출
    if target and target.operation == '+':
        find_selector(target.left, gate_idx, c, selector, direction)
        find_selector(target.right, gate_idx, c, selector, direction)

    # 곱셈 노드이면서 직접 값이 맞는 경우
    if gate.operation == '*' and target.evaluate() == c:
        selector[gate_idx] = 1

def gate_selector(gate_list, direction, transcript_table):
    """게이트에서 left 또는 right selector polynomial을 찾는 함수"""
    polynomial_table = []
    for c in transcript_table:
        selector = [0] * len(gate_list)
        for gate_idx, gate in enumerate(gate_list):
            find_selector(gate, gate_idx, c, selector, direction)
        polynomial_table.append(selector)
    return polynomial_table



In [116]:

# 실행 예시 (left selector 생성)
left_selector_table = gate_selector(gate_list, 'left', transcript_table)
print("Left Selector Table:")
left_selector_table

Left Selector Table:


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [117]:

# 실행 예시 (left selector 생성)
right_selector_table = gate_selector(gate_list, 'right', transcript_table)
print("Right Selector Table:")
right_selector_table

Right Selector Table:


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [118]:

# 실행 예시 (left selector 생성)
output_selector_table = gate_selector(gate_list, 'output', transcript_table)
print("output Selector Table:")
output_selector_table

output Selector Table:


[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]

In [119]:
def properties(transcript_table, gate_selector, omega):
    result = 0
    for idx, c in enumerate(transcript_table):
        result += c * gate_selector[idx][omega]
        
    return result
    

In [121]:
print(properties(transcript_table, left_selector_table, 0))
print(properties(transcript_table, left_selector_table, 1))
print(properties(transcript_table, left_selector_table, 2))

3
6
8


In [122]:
print(properties(transcript_table, right_selector_table, 0))
print(properties(transcript_table, right_selector_table, 1))
print(properties(transcript_table, right_selector_table, 2))

2
8
9


In [123]:
print(properties(transcript_table, output_selector_table, 0))
print(properties(transcript_table, output_selector_table, 1))
print(properties(transcript_table, output_selector_table, 2))

6
48
72


In [125]:
def master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, omega):
    left_properties = properties(transcript_table, left_selector_table, omega)
    right_properties = properties(transcript_table, right_selector_table, omega)
    output_properties = properties(transcript_table, output_selector_table, omega)
    
    return left_properties * right_properties - output_properties

In [127]:
print(master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, 0))
print(master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, 1))
print(master_polynomial(transcript_table, left_selector_table, right_selector_table, output_selector_table, 2))

0
0
0


In [128]:
def properties(transcript_table, gate_selector):
    """주어진 selector 테이블에 대해 property 값을 계산하는 함수"""
    def compute(omega):
        return sum(c * gate_selector[idx][omega] for idx, c in enumerate(transcript_table))
    return compute

def master_polynomial(transcript_table, selector_tables):
    """QAP의 master polynomial을 계산하는 클로저"""
    def compute(omega):
        left_properties = properties(transcript_table, selector_tables['left'])(omega)
        right_properties = properties(transcript_table, selector_tables['right'])(omega)
        output_properties = properties(transcript_table, selector_tables['output'])(omega)

        return left_properties * right_properties - output_properties

    return compute


In [130]:
selector_tables = {
    'left': left_selector_table,
    'right': right_selector_table,
    'output': output_selector_table
}

master_poly = master_polynomial(transcript_table, selector_tables)

# 특정한 omega 값으로 평가
print(master_poly(0))
print(master_poly(1))
print(master_poly(2))


0
0
0


In [131]:
def vanishing_polynomial(omegas):
    """주어진 selector 테이블에 대해 vanishing polynomial을 계산하는 함수"""
    def compute(x):
        return (x - omegas[0]) * (x - omegas[1]) * (x - omegas[2])
    
    return compute

In [288]:
class FiniteField:
    def __init__(self, value, prime):
        if prime <= 1:
            raise ValueError("Prime must be greater than 1")
        self.value = value % prime
        self.prime = prime
        
    def __add__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value + other.value, self.prime)
    
    def __sub__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value - other.value, self.prime)

    def __mul__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value * other.value, self.prime)
    
    def __neg__(self):
        return FiniteField(-self.value, self.prime)

    def __truediv__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        if other.value == 0:
            raise ZeroDivisionError("Cannot divide by zero in Finite Field")
        
        return FiniteField(self.value * pow(other.value, -1, self.prime), self.prime)
    
    def __eq__(self, other):
        return isinstance(other, FiniteField) and self.value == other.value and self.prime == other.prime
    
    def __hash__(self):
        """ Make FiniteField hashable so it can be used as a dictionary key """
        return hash((self.value, self.prime))

    def __repr__(self):
        return f"FiniteField({self.value}, {self.prime})"


class EllipticCurve:
    def __init__(self, a, b, prime):
        """타원 곡선 E: y^2 = x^3 + ax + b 정의"""
        self.a = FiniteField(a, prime)
        self.b = FiniteField(b, prime)
        self.prime = prime        
        self.infinity = None  # 무한대의 점 (Point at Infinity)
        self.points = [self.infinity]

        # Discriminant 체크 (4a^3 + 27b^2 != 0 이어야 타원 곡선)
        self.discriminant = 4 * self.a.value**3 + 27 * self.b.value**2
        if self.discriminant % prime == 0:
            raise ValueError("This curve is not elliptic (discriminant is zero)")

        # 유한체 위에서 점 찾기
        for x in range(prime):
            for y in range(prime):
                if self.is_point_on_curve(FiniteField(x, prime), FiniteField(y, prime)):
                    self.points.append((FiniteField(x, prime), FiniteField(y, prime)))

    def is_point_on_curve(self, x, y):
        """점 (x, y) 가 타원 곡선 위에 있는지 확인"""
        return (y * y).value == (x * x * x + self.a * x + self.b).value % self.prime
    
    def add_points(self, p1:(FiniteField, FiniteField), p2:(FiniteField, FiniteField)) -> FiniteField:
        if p1 not in self.points or p2 not in self.points:
            raise ValueError("Points are not on the curve")
        
        # 무한대의 점 처리
        if p1 is None:
            return p2
        if p2 is None:
            return p1     

        x1, y1 = p1
        x2, y2 = p2
        
        # Case 3: P와 Q가 서로 역원인 경우 (x 좌표가 같고, y 좌표가 반대인 경우)
        if x1 == x2 and y1 != y2:
            return None  # 무한대의 점 반환
        
        # Case 2: 점 두 배 연산 (Doubling, P == Q)
        if p1 == p2:
            if y1.value == 0:
                return None  # 접선이 무한대인 경우
            slope = (FiniteField(3, self.prime) * x1 * x1 + self.a) / (FiniteField(2, self.prime) * y1)
        else:
            if x1 == x2:
                return None  # 기울기가 정의되지 않음
            slope = (y2 - y1) / (x2 - x1)  # 기울기 계산
            
        # x3, y3 계산
        x3 = slope * slope - x1 - x2
        y3 = slope * (x1 - x3) - y1

        return (x3, y3)


    def __repr__(self):
        return f"EllipticCurve(y^2 = x^3 + {self.a.value}x + {self.b.value} over F_{self.prime})"


In [289]:
ecc = EllipticCurve(0, 1, 5)
print(ecc.points)

[None, (FiniteField(0, 5), FiniteField(1, 5)), (FiniteField(0, 5), FiniteField(4, 5)), (FiniteField(2, 5), FiniteField(2, 5)), (FiniteField(2, 5), FiniteField(3, 5)), (FiniteField(4, 5), FiniteField(0, 5))]


In [290]:
ecc.points

[None,
 (FiniteField(0, 5), FiniteField(1, 5)),
 (FiniteField(0, 5), FiniteField(4, 5)),
 (FiniteField(2, 5), FiniteField(2, 5)),
 (FiniteField(2, 5), FiniteField(3, 5)),
 (FiniteField(4, 5), FiniteField(0, 5))]

In [291]:
print(ecc.add_points(ecc.points[1], ecc.points[1]))
print(ecc.add_points(ecc.points[1], ecc.points[2]))

(FiniteField(0, 5), FiniteField(4, 5))
None


In [360]:
from collections import Counter

# https://crypto.stanford.edu/~dabo/papers/bfibe.pdf
# Identity-Based Encryption from the Weil Pairing - Appendix: Definition of the Weil pairing
class Divisor:
    def __init__(self, curve:EllipticCurve):
        self.curve = curve
        self.points = curve.points        
        
    def find_intersection(self, linear_function):
        # linear_function = (a, b, c) indicating ax + by + c
        zeros = []
        a, b, c = linear_function
        for point in self.points:
            if point == self.curve.infinity:
                continue
            
            x, y = point
            if a * x + b * y + c == FiniteField(0, self.curve.prime):
                zeros.append(point)
                
        return zeros

    def compute_divisor(self, linear_function):
        """ 주어진 선형 함수로부터 divisor를 계산하는 메서드 (접점 고려) """
        zeros = self.find_intersection(linear_function)
        divisor = Counter()

        # 접점 감지 로직 추가: 동일한 점이 여러 번 나오면 차수를 증가!
        for point in zeros:
            divisor[point] += 1  # 접점이면 차수가 2 이상이 될 수 있음

        # 무한대의 점을 divisor에 추가 (극점 차수는 3)
        if len(zeros) > 0:
            divisor[self.curve.infinity] = -3

        return divisor 
    
    def is_principal_divisor(self, divisor):
        """ Principal Divisor 여부를 확인하는 함수 """
        # 첫 번째 조건: 모든 계수의 합이 0이어야 함
        if sum(divisor.values()) != 0:
            return False

        # 두 번째 조건: 모든 점 덧셈 결과가 항등원(O)이어야 함
        sum_of_points = self.curve.infinity  # 항등원으로 초기화

        for point, count in divisor.items():
            for _ in range(count):  # 계수(Weight)만큼 반복
                sum_of_points = self.curve.add_points(sum_of_points, point)

        return sum_of_points == self.curve.infinity

    def is_equivalent_divisior(self, divisor_A, divisor_B):
        difference = divisor_A.copy()
        difference.subtract(divisor_B)
        
        return self.is_principal_divisor(difference)
    
    def evaluate_function(self, divisor, linear_function):
        """ 함수를 divisor에 대해서 평가하는 함수 """
        result = 1
        a, b, c = linear_function  # 선형 함수의 계수 추출

        for point, exponent in divisor.items():
            if point == self.curve.infinity:
                continue  # 무한대의 점은 연산에서 제외
            x, y = point

            value = a*x + b*y + c
            if value == 0:
                raise ValueError("Function evaluation resulted in zero, which may cause invalid pairing.")

            result *= value.value ** exponent  # f(P)^a_P 연산 수행

        return result

In [361]:
div = Divisor(ecc)
divisor = div.compute_divisor((FiniteField(1, 5), FiniteField(1, 5), FiniteField(1, 5)))
print(divisor)
print(div.is_principal_divisor(divisor))
print(div.is_equivalent_divisior(divisor, divisor))

Counter({(FiniteField(0, 5), FiniteField(4, 5)): 1, (FiniteField(2, 5), FiniteField(2, 5)): 1, (FiniteField(4, 5), FiniteField(0, 5)): 1, None: -3})
True
True


In [394]:
class WeilPairing:
    def __init__(self, curve:EllipticCurve, P: (FiniteField, FiniteField), Q: (FiniteField, FiniteField)):
        self.curve = curve
        self.P = P
        self.Q = Q
        
        if P not in curve.points or Q not in curve.points:
            raise ValueError("Points are not on the curve")
        
        self.div = Divisor(curve)  # Divisor 객체를 한 번만 생성하여 사용
        
    def generate_linear_function(self, point: (FiniteField, FiniteField)):
        """주어진 점을 통해 선형 함수를 생성하는 메서드"""
        if point == self.curve.infinity:
            raise ValueError("Point at Infinity is not allowed")
        
        x_P, y_P = point
        
        return (FiniteField(1, self.curve.prime), FiniteField(0, self.curve.prime), FiniteField(-x_P.value, self.curve.prime))
        
    def compute_pairing(self):
        """ Weil Pairing 계산 """
        linear_function_P = self.generate_linear_function(self.P)
        linear_function_Q = self.generate_linear_function(self.Q)

        divisor_P = self.div.compute_divisor(linear_function_P)
        divisor_Q = self.div.compute_divisor(linear_function_Q)
        
        # divisor에 대해 함수 평가
        numerator = self.div.evaluate_function(divisor_Q, linear_function_P)  
        denominator = self.div.evaluate_function(divisor_P, linear_function_Q)  

        #return numerator * pow(denominator, -1, self.curve.prime)
        return FiniteField(numerator, self.curve.prime**2) / FiniteField(denominator, self.curve.prime**2)
        

In [395]:
ecc.points

[None,
 (FiniteField(0, 5), FiniteField(1, 5)),
 (FiniteField(0, 5), FiniteField(4, 5)),
 (FiniteField(2, 5), FiniteField(2, 5)),
 (FiniteField(2, 5), FiniteField(3, 5)),
 (FiniteField(4, 5), FiniteField(0, 5))]

In [396]:
P = (FiniteField(0, 5), FiniteField(4, 5))
Q = (FiniteField(2, 5), FiniteField(2, 5))

weil = WeilPairing(ecc, P, Q)

weil.compute_pairing()

FiniteField(6, 25)